In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

from itertools import product

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Step 1: Generate the true class balance to be recovered

In [2]:
K = 3

# Generate the true class balance to be recovered
p_Y = np.random.random(K)
p_Y /= p_Y.sum()
p_Y

array([0.33627531, 0.06528622, 0.59843847])

### Step 2: Generate the true conditional probability tables (CPTs) for the LFs

Separate simple process here to keep simple (later merge this with the SPA generator).
Generate in terms of the _conditional accuracies_ (which is equivalent to the recall...):
$$
\alpha_{i,y',y} = P(\lambda_i = y' | Y = y)
$$

Note that this table should be normalized such that:
$$
\sum_{y'} \alpha_{i,y',y} = 1
$$

In [3]:
M = 10
alphas = []
for i in range(M):
    a = np.random.random((K,K))
    alphas.append( a @ np.diag(1 / a.sum(axis=0)) )
alpha = np.array(alphas)

assert np.all(np.abs(alpha.sum(axis=1) - 1) < 1e-5)
alpha[0]

array([[0.28359862, 0.23194285, 0.39931097],
       [0.36570757, 0.39197016, 0.31187131],
       [0.35069381, 0.376087  , 0.28881771]])

### Aside: Different tensor product approaches in Python

#### (1) Brute force

In [4]:
%%time
O_1 = np.zeros((M,M,M,K,K,K))
for i, j, k, y1, y2, y3 in product(range(M), range(M), range(M), range(K), range(K), range(K)):
    for y in range(K):
        O_1[i,j,k,y1,y2,y3] += alpha[i, y1, y] * alpha[j, y2, y] * alpha[k, y3, y]

CPU times: user 129 ms, sys: 817 µs, total: 130 ms
Wall time: 130 ms


#### (3) `np.einsum`

In [5]:
%time O_3 = np.einsum('aby,cdy,efy->acebdf', alpha, alpha, alpha)

CPU times: user 1.44 ms, sys: 496 µs, total: 1.94 ms
Wall time: 1.3 ms


In [6]:
np.mean(np.abs(O_1 - O_3))

4.096404928229565e-18

Now, trying a bilinear form:

In [7]:
%%time
Op_1 = np.zeros((M,M,M,K,K,K))
for i, j, k, y1, y2, y3 in product(range(M), range(M), range(M), range(K), range(K), range(K)):
    for y in range(K):
        Op_1[i,j,k,y1,y2,y3] += p_Y[y] * alpha[i, y1, y] * alpha[j, y2, y] * alpha[k, y3, y]

CPU times: user 495 ms, sys: 4.54 ms, total: 500 ms
Wall time: 136 ms


In [8]:
%time Op_3 = np.einsum('aby,cdy,efy,y->acebdf', alpha, alpha, alpha, p_Y)

CPU times: user 622 µs, sys: 118 µs, total: 740 µs
Wall time: 530 µs


In [9]:
np.mean(np.abs(Op_1 - Op_3))

2.1384522560651964e-18

### Step 3: Generate the _three-way_ overlaps tensor $O$ of conditionally-independent LFs

Now we can directly generate $O$.
By our conditional independence assumption, we have:
$$
P(\lambda_i = y', \lambda_j = y'' | Y = y) = \alpha_{i,y',y} \alpha_{j,y'',y}
$$

Thus we have:
$$
O_{i,j,y',y''} = \sum_y P(Y=y) \alpha_{i,y',y} \alpha_{j,y'',y}
$$

In [10]:
# Compute mask
mask = torch.ones((M,M,M,K,K,K)).byte()
for i, j, k in product(range(M), repeat=3):
    if len(set((i,j,k))) < 3:
        mask[i,j,k,:,:,:] = 0

In [11]:
%%time
O = np.einsum('aby,cdy,efy,y->acebdf', alpha, alpha, alpha, p_Y)
O = torch.from_numpy(O).float()
O[1-mask] = 0

CPU times: user 12.1 ms, sys: 980 µs, total: 13.1 ms
Wall time: 3.12 ms


In [12]:
# Compute observed labeling rates
O_l = torch.from_numpy(np.einsum('aby,y->ab', alpha, p_Y)).float()

### Step 4: Try to recover $\alpha$ given $P(Y=y)$

In [13]:
def get_loss(A, P, O, O_l):
    
    # Main constraint: match empirical pairwise overlaps matrix (entries O_ij for i != j)
    loss_1 = torch.norm((O - torch.einsum('aby,cdy,efy,y->acebdf', [A,A,A,P]))[mask])**2
    
    # Col-wise stochastic: \sum_y' P(\lf=y'|Y=y) = 1.0
    loss_2 = torch.norm(torch.sum(A, 1) - 1)**2
    
    # Row-wise constraint: match observed labeling rates P(\lf=y') = \sum_y P(Y=y) P(\lf=y'|Y=y)
    loss_3 = torch.norm(O_l - torch.einsum('aby,y->ab', [A,P]))**2
    
    return loss_1 + loss_2 + loss_3

def train_model(A, P, O, O_l, n_epochs=10, lr=0.01, print_every=1):
    optimizer = optim.SGD([A], lr=lr)
    
    for epoch in range(n_epochs):
        epoch_loss = 0.0
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass to calculate outputs
        loss = get_loss(A, P, O, O_l)

        # Backward pass to calculate gradients
        loss.backward()

        # Perform optimizer step
        optimizer.step()

        # Keep running sum of losses
        epoch_loss += loss.detach()

        # Report progress
        if epoch % print_every == 0:
            msg = f"[E:{epoch}]\tLoss: {epoch_loss:.8f}"
            print(msg)

In [33]:
A = nn.Parameter(torch.from_numpy(np.random.rand(M, K, K)).float()).float()
P = torch.from_numpy(p_Y).float()

train_model(A, P, O, O_l, n_epochs=10000, lr=0.01, print_every=1000)

print(f"Param estimation error: {np.mean(np.abs(A.detach().numpy() - alpha))}")

[E:0]	Loss: 486.86047363
[E:1000]	Loss: 0.02988081
[E:2000]	Loss: 0.01138565
[E:3000]	Loss: 0.00566233
[E:4000]	Loss: 0.00378250
[E:5000]	Loss: 0.00243895
[E:6000]	Loss: 0.00145715
[E:7000]	Loss: 0.00080929
[E:8000]	Loss: 0.00042357
[E:9000]	Loss: 0.00021229
Param estimation error: 0.0044183650801324715


In [34]:
A[0]

tensor([[0.2820, 0.2384, 0.3995],
        [0.3665, 0.3889, 0.3117],
        [0.3516, 0.3726, 0.2887]], grad_fn=<SelectBackward>)

In [35]:
alpha[0]

array([[0.28359862, 0.23194285, 0.39931097],
       [0.36570757, 0.39197016, 0.31187131],
       [0.35069381, 0.376087  , 0.28881771]])

In [36]:
A[0].sum(1)

tensor([0.9199, 1.0672, 1.0129], grad_fn=<SumBackward1>)

In [37]:
alpha[0].sum(1)

array([0.91485244, 1.06954904, 1.01559852])